In [21]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

end_date = datetime.today()
start_date_daily = datetime(2015, 1, 1)
start_date_hourly = end_date - timedelta(days=90)

tickers = {
    "bitcoin": "BTC-USD",
    "gold": "GC=F",
    "oil": "CL=F"
}

def download_data(ticker, start, end, interval):
    return yf.download(ticker, start=start, end=end, interval=interval, auto_adjust=True)

btc_daily = download_data(tickers["bitcoin"], start_date_daily, end_date, "1d")
gold_daily = download_data(tickers["gold"], start_date_daily, end_date, "1d")
oil_daily = download_data(tickers["oil"], start_date_daily, end_date, "1d")

btc_1h = download_data(tickers["bitcoin"], start_date_hourly, end_date, "1h")
gold_1h = download_data(tickers["gold"], start_date_hourly, end_date, "1h")
oil_1h = download_data(tickers["oil"], start_date_hourly, end_date, "1h")

btc_4h = download_data(tickers["bitcoin"], start_date_hourly, end_date, "4h")
gold_4h = download_data(tickers["gold"], start_date_hourly, end_date, "4h")
oil_4h = download_data(tickers["oil"], start_date_hourly, end_date, "4h")

btc_daily.to_csv("btc_daily.csv")
gold_daily.to_csv("gold_daily.csv")
oil_daily.to_csv("oil_daily.csv")

btc_1h.to_csv("btc_1h.csv")
gold_1h.to_csv("gold_1h.csv")
oil_1h.to_csv("oil_1h.csv")

btc_4h.to_csv("btc_4h.csv")
gold_4h.to_csv("gold_4h.csv")
oil_4h.to_csv("oil_4h.csv")

def prepare_combined_dataset(btc_df, gold_df, oil_df):
    df = btc_df[['Close']].rename(columns={'Close': 'Close_btc'})
    df['Close_gold'] = gold_df['Close']
    df['Close_oil'] = oil_df['Close']
    df.dropna(inplace=True)
    df['Target'] = df['Close_btc'].shift(-1)
    df.dropna(inplace=True)
    df["Day"] = np.arange(1, len(df)+1)
    return df

df_1h = prepare_combined_dataset(btc_1h, gold_1h, oil_1h)
df_4h = prepare_combined_dataset(btc_4h, gold_4h, oil_4h)
df_1d = prepare_combined_dataset(btc_daily, gold_daily, oil_daily)

# 💾 ذخیره‌سازی فایل‌های نهایی
df_1h.to_csv("df_1h.csv")
df_4h.to_csv("df_4h.csv")
df_1d.to_csv("df_1d.csv")


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [22]:
import pandas as pd

def load_and_clean_csv(filepath):
    df = pd.read_csv(filepath, skiprows=2)
    expected_cols = ['Datetime', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
    df.columns = expected_cols[:df.shape[1]]
    df['Datetime'] = pd.to_datetime(df['Datetime'])
    df.set_index('Datetime', inplace=True)
    return df

btc_1h = load_and_clean_csv("btc_1h_raw.csv")
gold_1h = load_and_clean_csv("gold_1h_raw.csv")
oil_1h = load_and_clean_csv("oil_1h_raw.csv")

btc_4h = load_and_clean_csv("btc_4h_direct.csv")
gold_4h = load_and_clean_csv("gold_4h_direct.csv")
oil_4h = load_and_clean_csv("oil_4h_direct.csv")

btc_1d = load_and_clean_csv("btc_daily_raw.csv")
gold_1d = load_and_clean_csv("gold_daily_raw.csv")
oil_1d = load_and_clean_csv("oil_daily_raw.csv")

btc_1h.info(), btc_1h.head(2)


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2157 entries, 2025-03-01 00:00:00+00:00 to 2025-05-29 20:00:00+00:00
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       2157 non-null   float64
 1   High       2157 non-null   float64
 2   Low        2157 non-null   float64
 3   Close      2157 non-null   float64
 4   Adj Close  2157 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 101.1 KB


(None,
                                    Open          High           Low  \
 Datetime                                                              
 2025-03-01 00:00:00+00:00  83832.046875  84510.898438  83794.234375   
 2025-03-01 01:00:00+00:00  84626.718750  84741.898438  83809.539062   
 
                                   Close  Adj Close  
 Datetime                                            
 2025-03-01 00:00:00+00:00  84330.851562          0  
 2025-03-01 01:00:00+00:00  83809.539062          0  )

In [23]:
def prepare_full_dataset(btc_df, gold_df, oil_df):
    btc = btc_df.add_prefix('btc_')
    gold = gold_df.add_prefix('gold_')
    oil = oil_df.add_prefix('oil_')

    df = btc.join([gold, oil], how='inner')
    df.dropna(inplace=True)
    df['Day'] = range(1, len(df) + 1)
    df['Target'] = df['btc_Close'].shift(-1)
    df.dropna(inplace=True)
    return df

df_1h = prepare_full_dataset(btc_1h, gold_1h, oil_1h)
df_4h = prepare_full_dataset(btc_4h, gold_4h, oil_4h)
df_1d = prepare_full_dataset(btc_1d, gold_1d, oil_1d)

df_1h.to_csv("df_1h.csv")
df_4h.to_csv("df_4h.csv")
df_1d.to_csv("df_1d.csv")

In [24]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import layers, models, metrics

df_1h = pd.read_csv("df_1h.csv")
df_4h = pd.read_csv("df_4h.csv")
df_1d = pd.read_csv("df_1d.csv")

param_grid_rf = {'n_estimators': [50], 'max_depth': [None]}
param_grid_mlp = {'hidden_layer_sizes': [(50,)], 'max_iter': [500]}
param_grid_dt = {'max_depth': [None]}
param_grid_knn = {'n_neighbors': [3]}
param_grid_ridge = {'alpha': [1.0]}

def perform_grid_search(model, param_grid, X_train, y_train):
    start_time = time.time()
    search = GridSearchCV(model, param_grid, cv=3, scoring='neg_mean_absolute_error', n_jobs=-1)
    search.fit(X_train, y_train)
    train_time = time.time() - start_time
    return search.best_estimator_, train_time

def train_ml(df, timeframe):
    X = df.drop(columns=["Target", "Datetime"])
    y = df["Target"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=False)

    models_dict = {
        "Ridge": (Ridge(), param_grid_ridge),
        "MLP": (MLPRegressor(), param_grid_mlp),
        "LinearRegression": (LinearRegression(), {}),
        "DecisionTree": (DecisionTreeRegressor(), param_grid_dt),
        "KNN": (KNeighborsRegressor(), param_grid_knn),
        "RandomForest": (RandomForestRegressor(), param_grid_rf),
    }

    results = []
    for name, (model, params) in models_dict.items():
        if params:
            best_model, train_time = perform_grid_search(model, params, X_train, y_train)
        else:
            start = time.time()
            model.fit(X_train, y_train)
            best_model = model
            train_time = time.time() - start

        start_pred = time.time()
        preds = best_model.predict(X_test)
        pred_time = time.time() - start_pred
        mae = mean_absolute_error(y_test, preds)
        r2 = r2_score(y_test, preds)
        acc = 1 - (mae / y_test.mean())

        results.append({
            "Model": name,
            "Timeframe": timeframe,
            "TrainTime": train_time,
            "PredictTime": pred_time,
            "MAE": mae,
            "R2": r2,
            "Accuracy": acc
        })
    return results

def sequence_maker(data, window_size, target_index):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:i+window_size])
        y.append(data[i+window_size, target_index])
    return np.array(X), np.array(y)

def train_rnn(X, y, tf, features):
    X_train, X_test, y_train, y_test = train_test_split(X, y.reshape(-1, 1), test_size=0.1, shuffle=False)

    model = models.Sequential([
        layers.Input(shape=(20, features)),
        layers.SimpleRNN(20, return_sequences=True),
        layers.SimpleRNN(20),
        layers.Dense(1)
    ])
    model.compile(
        loss='mean_squared_error',
        optimizer='adam',
        metrics=[
            metrics.MeanSquaredError(name='mse'),
            metrics.MeanAbsoluteError(name='mae'),
            metrics.MeanAbsolutePercentageError(name='mape'),
            metrics.RootMeanSquaredError(name='rmse'),
            metrics.R2Score(name='r2')
        ]
    )

    start_train = time.time()
    model.fit(X_train, y_train, epochs=30, batch_size=128, validation_data=(X_test, y_test), verbose=0)
    train_time = time.time() - start_train

    start_pred = time.time()
    results = model.evaluate(X_test, y_test, verbose=0)
    pred_time = time.time() - start_pred

    return {
        "Model": "RNN",
        "Timeframe": tf,
        "TrainTime": train_time,
        "PredictTime": pred_time,
        "MAE": results[2],
        "R2": results[5],
        "Accuracy": 1 - (results[2] / np.mean(y_test))
    }

def run_rnn(df, tf):
    df = df.drop(columns=["Datetime"])
    target_col = [col for col in df.columns if col.endswith("btc_Close") or col == "Close_btc"]
    target_col = target_col[0] if target_col else "Close_btc"
    target_index = df.columns.get_loc(target_col)
    scaler = MinMaxScaler()
    scaled = scaler.fit_transform(df)
    X, y = sequence_maker(scaled, 20, target_index)
    return train_rnn(X, y, tf, features=X.shape[2])

results = []
for df, tf in zip([df_1h, df_4h, df_1d], ["1h", "4h", "1d"]):
    results.extend(train_ml(df, tf))
    results.append(run_rnn(df, tf))

df_results = pd.DataFrame(results)
print(df_results)

C:\Users\USER\anaconda3\envs\tf-env\Lib\site-packages\sklearn\linear_model\_ridge.py:215: LinAlgWarning: Ill-conditioned matrix (rcond=3.75887e-21): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\USER\anaconda3\envs\tf-env\Lib\site-packages\sklearn\linear_model\_ridge.py:215: LinAlgWarning: Ill-conditioned matrix (rcond=1.33231e-21): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\USER\anaconda3\envs\tf-env\Lib\site-packages\sklearn\linear_model\_ridge.py:215: LinAlgWarning: Ill-conditioned matrix (rcond=2.83966e-22): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


               Model Timeframe  TrainTime  PredictTime           MAE  \
0              Ridge        1h   3.140098     0.000000  6.056013e+01   
1                MLP        1h   1.719354     0.000000  3.325540e+04   
2   LinearRegression        1h   0.028105     0.002005  6.057268e+01   
3       DecisionTree        1h   1.663918     0.000000  2.552671e+03   
4                KNN        1h   1.562476     0.061470  1.273849e+04   
5       RandomForest        1h   2.684246     0.000000  1.743380e+03   
6                RNN        1h  10.086232     0.094985  5.757501e-02   
7              Ridge        4h   1.612336     0.000000  1.456735e+02   
8                MLP        4h   0.153385     0.015622  1.361052e+07   
9   LinearRegression        4h   0.000000     0.000000  1.455550e+02   
10      DecisionTree        4h   0.204224     0.000000  8.945909e+02   
11               KNN        4h   0.073953     0.002347  1.873523e+04   
12      RandomForest        4h   0.310917     0.012004  1.132425

In [25]:
import pandas as pd

df_1h = pd.read_csv("df_1h.csv")
df_4h = pd.read_csv("df_4h.csv")
df_1d = pd.read_csv("df_1d.csv")

def process_for_classification(df):
    df = df.drop(columns=["Target"])
    df["Benefit"] = df["btc_Close"] - df["btc_Open"]
    df["Target"] = (df["Benefit"] > 0).astype(int)
    df["Target"] = df["Target"].shift(-1)
    df = df.dropna().reset_index(drop=True)
    return df

df_1h_cls = process_for_classification(df_1h)
df_4h_cls = process_for_classification(df_4h)
df_1d_cls = process_for_classification(df_1d)

df_1h_cls.to_csv("df_1h_class.csv", index=False)
df_4h_cls.to_csv("df_4h_class.csv", index=False)
df_1d_cls.to_csv("df_1d_class.csv", index=False)

In [26]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import layers, models, metrics

df_1h = pd.read_csv("df_1h_class.csv")
df_4h = pd.read_csv("df_4h_class.csv")
df_1d = pd.read_csv("df_1d_class.csv")

param_grid_rf = {'n_estimators': [50], 'max_depth': [None]}
param_grid_mlp = {'hidden_layer_sizes': [(50,)], 'max_iter': [500]}
param_grid_dt = {'max_depth': [None]}
param_grid_knn = {'n_neighbors': [3]}

def perform_grid_search(model, param_grid, X_train, y_train):
    start_time = time.time()
    search = GridSearchCV(model, param_grid, cv=3, scoring='accuracy', n_jobs=-1)
    search.fit(X_train, y_train)
    train_time = time.time() - start_time
    return search.best_estimator_, train_time

def train_classification_models(df, timeframe):
    X = df.drop(columns=["Target", "Datetime"])
    y = df["Target"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=False)

    models_dict = {
        "LogisticRegression": (LogisticRegression(max_iter=500), {}),
        "MLP": (MLPClassifier(), param_grid_mlp),
        "DecisionTree": (DecisionTreeClassifier(), param_grid_dt),
        "KNN": (KNeighborsClassifier(), param_grid_knn),
        "RandomForest": (RandomForestClassifier(), param_grid_rf),
    }

    results = []
    for name, (model, params) in models_dict.items():
        if params:
            best_model, train_time = perform_grid_search(model, params, X_train, y_train)
        else:
            start = time.time()
            model.fit(X_train, y_train)
            best_model = model
            train_time = time.time() - start

        start_pred = time.time()
        preds = best_model.predict(X_test)
        pred_time = time.time() - start_pred

        results.append({
            "Model": name,
            "Timeframe": timeframe,
            "TrainTime": train_time,
            "PredictTime": pred_time,
            "Accuracy": accuracy_score(y_test, preds),
            "Precision": precision_score(y_test, preds, zero_division=0),
            "Recall": recall_score(y_test, preds, zero_division=0),
            "F1": f1_score(y_test, preds, zero_division=0)
        })
    return results

def sequence_maker(data, window_size, target_index):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:i+window_size])
        y.append(data[i+window_size, target_index])
    return np.array(X), np.array(y)

def train_rnn_classifier(X, y, tf, features):
    X_train, X_test, y_train, y_test = train_test_split(X, y.reshape(-1, 1), test_size=0.1, shuffle=False)

    model = models.Sequential([
        layers.Input(shape=(20, features)),
        layers.SimpleRNN(20, return_sequences=True),
        layers.SimpleRNN(20),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(
        loss='binary_crossentropy',
        optimizer='adam',
        metrics=[
            metrics.BinaryAccuracy(name='accuracy'),
            metrics.Precision(name='precision'),
            metrics.Recall(name='recall')
        ]
    )

    start_train = time.time()
    model.fit(X_train, y_train, epochs=30, batch_size=128, validation_data=(X_test, y_test), verbose=0)
    train_time = time.time() - start_train

    start_pred = time.time()
    eval_result = model.evaluate(X_test, y_test, verbose=0)
    pred_time = time.time() - start_pred

    return {
        "Model": "RNN",
        "Timeframe": tf,
        "TrainTime": train_time,
        "PredictTime": pred_time,
        "Accuracy": eval_result[1],
        "Precision": eval_result[2],
        "Recall": eval_result[3],
        "F1": 2 * eval_result[2] * eval_result[3] / (eval_result[2] + eval_result[3] + 1e-10)
    }

def run_rnn(df, tf):
    df = df.drop(columns=["Datetime"])
    target_col = "Target"
    target_index = df.columns.get_loc(target_col)
    scaler = MinMaxScaler()
    scaled = scaler.fit_transform(df)
    X, y = sequence_maker(scaled, 20, target_index)
    return train_rnn_classifier(X, y, tf, features=X.shape[2])

results = []
for df, tf in zip([df_1h, df_4h, df_1d], ["1h", "4h", "1d"]):
    results.extend(train_classification_models(df, tf))
    results.append(run_rnn(df, tf))

df_results_cls = pd.DataFrame(results)
print(df_results_cls)

                 Model Timeframe  TrainTime  PredictTime  Accuracy  Precision  \
0   LogisticRegression        1h   0.041258     0.006006  0.524823   0.592593   
1                  MLP        1h   0.249256     0.000000  0.489362   0.500000   
2         DecisionTree        1h   0.270131     0.000000  0.482270   0.496124   
3                  KNN        1h   0.294130     0.015631  0.496454   0.510204   
4         RandomForest        1h   0.679449     0.016192  0.489362   0.500000   
5                  RNN        1h   6.584813     0.086984  0.553957   0.569444   
6   LogisticRegression        4h   0.015626     0.000000  0.514286   0.533333   
7                  MLP        4h   0.100038     0.000000  0.542857   0.542857   
8         DecisionTree        4h   0.152770     0.000000  0.571429   0.571429   
9                  KNN        4h   0.227700     0.008018  0.428571   0.454545   
10        RandomForest        4h   0.298560     0.006505  0.428571   0.000000   
11                 RNN      

In [27]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import layers, models, metrics

df_1h = pd.read_csv("df_1h_class.csv")
df_4h = pd.read_csv("df_4h_class.csv")
df_1d = pd.read_csv("df_1d_class.csv")


param_grid_rf = {'n_estimators': [50], 'max_depth': [None]}
param_grid_mlp = {'hidden_layer_sizes': [(50,)], 'max_iter': [500]}
param_grid_dt = {'max_depth': [None]}
param_grid_knn = {'n_neighbors': [3]}

def perform_grid_search(model, param_grid, X_train, y_train):
    start_time = time.time()
    search = GridSearchCV(model, param_grid, cv=3, scoring='accuracy', n_jobs=-1)
    search.fit(X_train, y_train)
    train_time = time.time() - start_time
    return search.best_estimator_, train_time

def train_classification_models(df, timeframe):
    X = df.drop(columns=["Target", "Datetime"])
    y = df["Target"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=False)

    models_dict = {
        "LogisticRegression": (LogisticRegression(max_iter=500), {}),
        "MLP": (MLPClassifier(), param_grid_mlp),
        "DecisionTree": (DecisionTreeClassifier(), param_grid_dt),
        "KNN": (KNeighborsClassifier(), param_grid_knn),
        "RandomForest": (RandomForestClassifier(), param_grid_rf),
    }

    results = []
    for name, (model, params) in models_dict.items():
        if params:
            best_model, train_time = perform_grid_search(model, params, X_train, y_train)
        else:
            start = time.time()
            model.fit(X_train, y_train)
            best_model = model
            train_time = time.time() - start

        start_pred = time.time()
        preds = best_model.predict(X_test)
        pred_time = time.time() - start_pred

        results.append({
            "Model": name,
            "Timeframe": timeframe,
            "TrainTime": train_time,
            "PredictTime": pred_time,
            "Accuracy": accuracy_score(y_test, preds),
            "Precision": precision_score(y_test, preds, zero_division=0),
            "Recall": recall_score(y_test, preds, zero_division=0),
            "F1": f1_score(y_test, preds, zero_division=0)
        })
    return results

def sequence_maker(data, window_size, target_index):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:i+window_size])
        y.append(data[i+window_size, target_index])
    return np.array(X), np.array(y)

def train_rnn_classifier(X, y, tf, features):
    from sklearn.metrics import precision_score, recall_score, f1_score

    X_train, X_test, y_train, y_test = train_test_split(X, y.reshape(-1, 1), test_size=0.1, shuffle=False)

    model = models.Sequential([
        layers.Input(shape=(20, features)),
        layers.SimpleRNN(20, return_sequences=True),
        layers.SimpleRNN(20),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(
        loss='binary_crossentropy',
        optimizer='adam',
        metrics=[
            metrics.BinaryAccuracy(name='accuracy'),
            metrics.Precision(name='precision'),
            metrics.Recall(name='recall')
        ]
    )

    start_train = time.time()
    model.fit(X_train, y_train, epochs=30, batch_size=128, validation_data=(X_test, y_test), verbose=0)
    train_time = time.time() - start_train

    start_pred = time.time()
    y_pred_prob = model.predict(X_test)
    y_pred_class = (y_pred_prob > 0.5).astype(int)
    pred_time = time.time() - start_pred

    preds_df = pd.DataFrame({
        'True': y_test.flatten(),
        'Predicted': y_pred_class.flatten(),
        'Probability': y_pred_prob.flatten()
    })
    preds_df.to_csv(f"rnn_preds_{tf}.csv", index=False)

    acc = np.mean(y_pred_class.flatten() == y_test.flatten())
    prec = precision_score(y_test, y_pred_class, zero_division=0)
    rec = recall_score(y_test, y_pred_class, zero_division=0)
    f1 = f1_score(y_test, y_pred_class, zero_division=0)

    return {
        "Model": "RNN",
        "Timeframe": tf,
        "TrainTime": train_time,
        "PredictTime": pred_time,
        "Accuracy": acc,
        "Precision": prec,
        "Recall": rec,
        "F1": f1
    }

def run_rnn(df, tf):
    df = df.drop(columns=["Datetime"])
    target_col = "Target"
    target_index = df.columns.get_loc(target_col)
    scaler = MinMaxScaler()
    scaled = scaler.fit_transform(df)
    X, y = sequence_maker(scaled, 20, target_index)
    return train_rnn_classifier(X, y, tf, features=X.shape[2])

results = []
for df, tf in zip([df_1h, df_4h, df_1d], ["1h", "4h", "1d"]):
    results.extend(train_classification_models(df, tf))
    results.append(run_rnn(df, tf))

df_results_cls = pd.DataFrame(results)
print(df_results_cls)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step
                 Model Timeframe  TrainTime  PredictTime  Accuracy  Precision  \
0   LogisticRegression        1h   0.031253     0.000000  0.524823   0.592593   
1                  MLP        1h   0.146653     0.000000  0.482270   0.495868   
2         DecisionTree        1h   0.101131     0.000000  0.482270   0.496124   
3                  KNN        1h   0.136436     0.008507  0.496454   0.510204   
4         RandomForest        1h   0.500797     0.015625  0.560284   0.559524   
5                  RNN        1h   6.583073     0.465921  0.589928   0.641509   
6   LogisticRegression        4h   0.031700     0.000507  0.514286   0.533333   
7                  MLP        4h   0.122909     0.000000  0.514286   0.531250   
8         DecisionTree        4h   0.073548     0.000000  0.514286   0.550000   
9                  KNN        4h   0.047812     0.007704  0.428571   0.4545